In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
import random

# prepare data

In [2]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

unlabeld_index = np.ones(y_train.shape, np.bool)

N = 20
for i in range(10):
  idx = np.where(y_train == i)[0][:N]
  unlabeld_index[idx] = 0

x_unlabeld = x_train[np.where(unlabeld_index)[0], ...]

x_train = x_train[np.where(~unlabeld_index)[0], ...]
y_train = y_train[np.where(~unlabeld_index)[0], ...]

170498071/170498071 [==============================] - 18s 0us/step


<ipython-input-2-2c2ed39229d7>:3: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  unlabeld_index = np.ones(y_train.shape, np.bool)


In [3]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_unlabeld = x_unlabeld.astype('float32') / 255.0

In [4]:
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

In [6]:
# examples of categorical crossentropy
cce = keras.losses.CategoricalCrossentropy()

# a labeled data from the second class
y_true = [[0, 1, 0, 0]]
y_pred = [[0.05, 0.95, 0, 0]]
print(cce(y_true, y_pred).numpy())

# an ulabeled data
y_true = [[0, 0, 0, 0]]
y_pred = [[0.05, 0.95, 0, 0]]
print(cce(y_true, y_pred).numpy())

# another ulabeled data
y_true = [[0, 0, 0, 0]]
y_pred = [[0.1, 0.4, 0.3, 0.2]]
print(cce(y_true, y_pred).numpy())

0.051293306
0.0
0.0


# Part A

In [5]:
model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=x_train[0].shape))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D())
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 10, 10, 64)        3

In [6]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(x_test, y_test)
)

Epoch 1/20
4/4 [==============================] - 10s 383ms/step - loss: 3.1646 - accuracy: 0.0750 - val_loss: 2.3022 - val_accuracy: 0.1027
Epoch 2/20
4/4 [==============================] - 0s 161ms/step - loss: 1.7429 - accuracy: 0.3800 - val_loss: 2.2946 - val_accuracy: 0.1039
Epoch 3/20
4/4 [==============================] - 0s 159ms/step - loss: 1.2589 - accuracy: 0.5350 - val_loss: 2.2842 - val_accuracy: 0.1122
Epoch 4/20
4/4 [==============================] - 0s 155ms/step - loss: 1.0160 - accuracy: 0.6850 - val_loss: 2.2765 - val_accuracy: 0.1600
Epoch 5/20
4/4 [==============================] - 1s 175ms/step - loss: 0.7904 - accuracy: 0.7700 - val_loss: 2.2725 - val_accuracy: 0.1451
Epoch 6/20
4/4 [==============================] - 1s 220ms/step - loss: 0.5779 - accuracy: 0.8550 - val_loss: 2.2678 - val_accuracy: 0.1470
Epoch 7/20
4/4 [==============================] - 1s 219ms/step - loss: 0.4057 - accuracy: 0.9350 - val_loss: 2.2640 - val_accuracy: 0.1612
Epoch 8/20
4/4 [===

# Part B

In [5]:
x_rotated = np.zeros_like(x_unlabeld)
y_rotated = np.zeros((x_unlabeld.shape[0],))
for i in range(x_rotated.shape[0]):
    k = random.randint(0, 3)
    x_rotated[i] = np.rot90(x_unlabeld[i], k)
    y_rotated[i] = k

y_rotated = keras.utils.to_categorical(y_rotated, num_classes=4)

In [8]:
model = keras.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=x_train[0].shape))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D())
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.MaxPool2D())
model.add(layers.Flatten())
model.add(layers.Dense(4, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_2 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 conv2d_7 (Conv2D)           (None, 10, 10, 64)       

In [9]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.fit(
    x_rotated, y_rotated,
    epochs=20,
    batch_size=64,
    shuffle=True
)

Epoch 1/20
779/779 [==============================] - 5s 6ms/step - loss: 1.1743 - accuracy: 0.5062
Epoch 2/20
779/779 [==============================] - 5s 6ms/step - loss: 1.0438 - accuracy: 0.5725
Epoch 3/20
779/779 [==============================] - 5s 6ms/step - loss: 0.9663 - accuracy: 0.6105
Epoch 4/20
779/779 [==============================] - 5s 6ms/step - loss: 0.8910 - accuracy: 0.6415
Epoch 5/20
779/779 [==============================] - 5s 6ms/step - loss: 0.8297 - accuracy: 0.6688
Epoch 6/20
779/779 [==============================] - 5s 6ms/step - loss: 0.7774 - accuracy: 0.6946
Epoch 7/20
779/779 [==============================] - 5s 6ms/step - loss: 0.7316 - accuracy: 0.7118
Epoch 8/20
779/779 [==============================] - 5s 6ms/step - loss: 0.6909 - accuracy: 0.7310
Epoch 9/20
779/779 [==============================] - 5s 6ms/step - loss: 0.6473 - accuracy: 0.7492
Epoch 10/20
779/779 [==============================] - 5s 6ms/step - loss: 0.6130 - accuracy: 0.7635

In [10]:
model_2 = keras.Model(inputs=model.inputs, outputs=layers.Dense(10, activation='softmax')(model.layers[-2].output))

model_2.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4_input (InputLayer)  [(None, 32, 32, 3)]      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 30, 30, 32)        896       
                                                                 
 conv2d_5 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_2 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 12, 12, 64)        18496 

In [11]:
model_2.compile(
    loss='categorical_crossentropy',
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['accuracy']
)

model_2.fit(
    x_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(x_test, y_test)
)

Epoch 1/20
4/4 [==============================] - 1s 313ms/step - loss: 2.6799 - accuracy: 0.1250 - val_loss: 2.7197 - val_accuracy: 0.0932
Epoch 2/20
4/4 [==============================] - 1s 219ms/step - loss: 2.5771 - accuracy: 0.1500 - val_loss: 2.6918 - val_accuracy: 0.0968
Epoch 3/20
4/4 [==============================] - 1s 222ms/step - loss: 2.4695 - accuracy: 0.1700 - val_loss: 2.6675 - val_accuracy: 0.1019
Epoch 4/20
4/4 [==============================] - 1s 219ms/step - loss: 2.3794 - accuracy: 0.1950 - val_loss: 2.6424 - val_accuracy: 0.1070
Epoch 5/20
4/4 [==============================] - 1s 174ms/step - loss: 2.3077 - accuracy: 0.2200 - val_loss: 2.6177 - val_accuracy: 0.1115
Epoch 6/20
4/4 [==============================] - 1s 173ms/step - loss: 2.2348 - accuracy: 0.2300 - val_loss: 2.5935 - val_accuracy: 0.1175
Epoch 7/20
4/4 [==============================] - 1s 170ms/step - loss: 2.1851 - accuracy: 0.2600 - val_loss: 2.5698 - val_accuracy: 0.1238
Epoch 8/20
4/4 [====

# Part C

In [14]:
x_train_concat = np.concatenate((x_train, x_rotated), axis=0)

y_train_classification = np.concatenate((y_train, np.zeros((x_rotated.shape[0], 10))), axis=0)
y_train_rotation = np.concatenate((np.array([[1, 0, 0, 0]] * x_train.shape[0]), y_rotated), axis=0) # I used [1, 0, 0, 0] (is 0) for the rotation of labeled data because they are not rotated

In [23]:
input = layers.Input(shape=x_train_concat[0].shape)
x = layers.Conv2D(32, (3, 3), activation='relu')(input)
x = layers.Conv2D(32, (3, 3), activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.MaxPool2D()(x)
out = layers.Flatten()(x)
base_model = keras.Model(inputs=input, outputs=out)

classification_layer = layers.Dense(10, activation='softmax', name='classification')(base_model.outputs[0])
rotation_layer = layers.Dense(4, activation='softmax', name='rotation')(base_model.outputs[0])

model = keras.Model(inputs=base_model.inputs, outputs=[classification_layer, rotation_layer])

model.summary()

Model: "model_15"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_28 (Conv2D)             (None, 30, 30, 32)   896         ['input_8[0][0]']                
                                                                                                  
 conv2d_29 (Conv2D)             (None, 28, 28, 32)   9248        ['conv2d_28[0][0]']              
                                                                                                  
 batch_normalization_14 (BatchN  (None, 28, 28, 32)  128         ['conv2d_29[0][0]']              
 ormalization)                                                                             

In [24]:
model.compile(
    loss={
        'classification': 'categorical_crossentropy',
        'rotation': 'categorical_crossentropy'
    },
    optimizer=keras.optimizers.Adam(learning_rate=1e-4, decay=1e-4 / 20),
    metrics=['accuracy'],
    loss_weights={
        'classification': 100,
        'rotation': 1
    }
)

model.fit(
    x_train_concat, [y_train_classification, y_train_rotation],
    batch_size=64,
    epochs=20,
    validation_data=(x_test, {'classification': y_test})
)

Epoch 1/20
782/782 [==============================] - 7s 8ms/step - loss: 3.0603 - classification_loss: 0.0126 - rotation_loss: 1.7988 - classification_accuracy: 0.0551 - rotation_accuracy: 0.2663 - val_loss: 295.4293 - val_classification_loss: 2.9543 - val_rotation_loss: 0.0000e+00 - val_classification_accuracy: 0.1065 - val_rotation_accuracy: 0.0000e+00
Epoch 2/20
782/782 [==============================] - 6s 8ms/step - loss: 2.8126 - classification_loss: 0.0123 - rotation_loss: 1.5788 - classification_accuracy: 0.0515 - rotation_accuracy: 0.2622 - val_loss: 310.0349 - val_classification_loss: 3.1003 - val_rotation_loss: 0.0000e+00 - val_classification_accuracy: 0.0989 - val_rotation_accuracy: 0.0000e+00
Epoch 3/20
782/782 [==============================] - 6s 8ms/step - loss: 2.7185 - classification_loss: 0.0123 - rotation_loss: 1.4901 - classification_accuracy: 0.0507 - rotation_accuracy: 0.2595 - val_loss: 305.9677 - val_classification_loss: 3.0597 - val_rotation_loss: 0.0000e+00 